In [81]:
from numpy import *
from scipy.optimize import fsolve
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

**Question 1 [16]**

A liquid phase reaction where component $A$ is converted to products is studied for *ADIABATIC* operation. The inlet temperature of a stream of pure $A$ (with $Q = 2~L.s^{-1}$ and $C_{A_{0}} = 0.1~mol.L^{-1}$ is $T_{0} = 300~K$

In figure 1 below, $-r_{A} (\dfrac{mol}{L.s})$ is given as a function of the conversion of $A$. Answer the following questions:

In [82]:
Q   = 2/1000 #m3/s
CAo =  0.1e3 #mol/m3
FAo =  Q*CAo
To  =    300 #K

**a) What is the maximum conversion of $A$ that can be obtained in an adiabatic plug flow reactor (PFR)? Explain your answer**

81% When the rate gets to zero, this is the maximum conversion at that equilibrium point. Although there are values past that for a reversible reaction, the reactor is adiabatic and exothermic (from the shape of the graph) and reaction is pushed to the right. 

**b) What is the maximum conversion that can be obtained in an adiabatic CSTR? Explain your answer**

Also 81%. Only difference is that more catalyst will need to be used for the CSTR

Additional comments: although graph shows possibility of reverse reaction (rate goes lower than zero), it doesn't happen because this reaction happens adiabatically.

------------------------------------------------------------

*An adiabatic PFR is sized to convert $70\%$ of component $A$. The outlet temperature from this reactor is $T = 477~K$*

**c) Draw a semi-qualitative sketch of conversion of $A$ (y-axis) vs reactor volume (x-axis) profile in this reactor. Clearly motivate the shape of your plot. Give as much quantitative information as possible. (no integration required!)**

<img src="Qc.JPG" width="850" />

Increasing slope to X at $-r_{A_{max}}$ inflection point at X =$54\%$, decreasing slope to $70\%$ (not zero slope at $70\%$).

Basic principles is that the rate will be quicker and the final max conversion will be lower. The point of inflection will happen at a lower spot than the inflection for the $80\%$ conversion graph.

--------------------------------------------------------------

*Two adiabatic CSTRs in series must be used to achieve a conversion of $70\%$ while minimizing the total volume of the reactor system (i.e. minimize $V_{CSTR1} + V_{CSTR2}$).*

**d) Specify the respective volumes of the two CSTRs. (Still overall adiabatic operation, so no cooling or heating between the two reactors)**

<img src="Qd.JPG" width="850" />

CSTR design equation:
\begin{align}
-r_{A} &= \dfrac{F_{A_{0}} - F_{A}}{V} \nonumber \\
&= \dfrac{F_{A_{0}} - F_{A_{0}}(1 - \text{X})}{V} \nonumber \\
-r_{A} &= \dfrac{F_{A_{0}}\text{X}}{V} \nonumber \\
 V &= \dfrac{F_{A_{0}}\text{X}}{-r_{A}} \nonumber 
\end{align}

From the graph, the maximum rate, $-r_{A}= 0.0054$ is at X$=0.54$

In [83]:
V1cstr = FAo*0.54/0.0054
V1cstr

20.0

In [84]:
#FA out of CSTR1 = FAo into CSTR2
FAo1 = FAo*(1 - 0.54)
FAo1

0.092

From the graph, at $70\%$ conversion, the rate is $0.0037$

In [85]:
FA = FAo*(1-.7)
FA

0.06000000000000001

In [86]:
V2cstr = (FAo1 - FA)/0.0037
V2cstr

8.648648648648646

----------------------------------------

**e) What is the inlet temperature to the second CSTR in this sequence?**

We know that the reaction is non-isothermal and adiabatic, so we can assume a constant mass based $C_{p}^{'}$. We therefore have the following straight line relationship between $T_{out}$ and $X_{A}$:
\begin{equation}
T_{out} = T_{in} + \dfrac{-\Delta H_{RX_{j}}^{\circ}}{C_{p_{A}}}X_{A} \nonumber
\end{equation}

We don't need the individual enthalpy and Cp but we can calculate the gradient and make use of that.

In [87]:
T = 477 #given exit temperature at equilibrium conversion of 
        # 70% for PFR

In [88]:
Mcstr = (T - To)/0.7 
Mcstr

252.85714285714286

Now that we have the gradient, we can calculate the exit temperature for the first CSTR at $X=54\%$

In [89]:
T1 = To + Mcstr*0.54
T1 #which will be the same as the inlet Temperature for CSTR2

436.54285714285714

<img src="Fig.1 Rate of Disappearance.JPG" width="850" />

-------------------------------------------------------------

**Question 2 [18]**

The irreversible gas phase reaction given below is allowed to take place in an *adiabatically* operated packed bed reactor (PBR).
$$
\begin{equation}
A \rightarrow B + C
\end{equation}
$$


Some of the information on the reaction and the reactor is given in the table below:

<img src="Q2 Table.JPG" width="850" />

In [90]:
FAo     =       0.11 #kmol.s
FBo     =          0
FCo     =          0
Po      =        300 #kPa abs (kJ/m3)
To      = 150+273.15 #K
dHrx    =     -30000 #kJ/Kmol
Kerg    =        0.4 #kPa/kg
koprime =       1000 #m3/kg.s
Ea      =      45000 #kJ/kmol
CpA     =        200 #kJ/(kmol.K)
dCprx   =          0 #kJ/(kmol.K)
R       =      8.314 #kJ/(kmol.K)


*Due to downstream processing requirements, the pressure at the outlet of the reactor may not be less than $150~kPa_{abs}$*

**a) What is the *maximum conversion* that can be achieved in this reactor? Also give the corresponding amount of catalyst (𝑾) in the reactor to achieve this conversion. [11]**

In [91]:
#P = 150 #kPa abs

In [110]:
W = 203.2 #kg - the amount varied until P = 150kPa
M = -dHrx/CpA #gradient for energy balance
def Q2a(var,w):
    FA, FB, FC, P   = var
    Ftot   = FA + FB + FC
    X      = (FAo - FA)/FAo
    T      = To + M*X
    kprime = koprime*exp(-Ea/(R*T))
    Q      = Ftot*R*T/P
    rA     = -kprime*FA/Q
    rB     = -rA
    rC     = -rA
    dFAdW  = rA
    dFBdW  = rB
    dFCdW  = rC
    dPdW   = -Kerg*(Po/P)*(Ftot/FAo)*(T/To)
    return[dFAdW,dFBdW,dFCdW,dPdW]       

In [111]:
wspan = linspace(0,W,100)
init = (FAo,0,0,Po)
sols = odeint(Q2a,init,wspan)

In [112]:
FA,FB,FC,P = sols.T

XA = (FAo - FA[-1])/FAo

print('conversion=', XA*100,'%')
print('W = ',W,'kg catalyst')  
print('P =',P[-1],'kPa')

conversion= 52.50556543291963 %
W =  203.2 kg catalyst
P = 150.1250426613858 kPa


**b) The tube diameter of the reactor is increased by 10%. Determine the maximum conversion that can be obtained in this new reactor. Once again give the corresponding catalyst mass.[7]**

Assuming Turbulent flow for the Ergun equation:

\begin{equation}
			\frac{dP}{dz} = -\dfrac{1.75G^{2}}{\rho_{\circ}d_{p}}\left(\frac{1 - \epsilon}{\epsilon}\right)\left(\dfrac{P_{\circ}}{P}\frac{F_{T}}{F_{T_{\circ}}}\frac{T}{T_{\circ}}\right)
\end{equation}


$K_{Ergun}$ units are in $\dfrac{Pressure}{m}$

		

From $ \dfrac{dP}{dz} $, $ dz = \dfrac{dW}{\rho_{B}*A_{across}} $
		

Where $ A_{across} = \pi\frac{D_{tube}^{2}}{4}$ therefore substituting $dz$ into equation (1) above in terms of $\frac{dP}{dW}  $:
$$
\begin{equation}
		\frac{dP}{dW} = -\dfrac{1.75}{\rho_{\circ}d_{p}}\left(\frac{1 - \epsilon}{\epsilon}\right)\dfrac{G^{2}}{\rho_{B}A_{across}}\left(\dfrac{P_{\circ}}{P}\frac{F_{T}}{F_{T_{\circ}}}\frac{T}{T_{\circ}}\right)
\end{equation}
$$		

$K_{Ergun}'$ units are in $\dfrac{Pressure}{kg}$ as given, So: 
$ $
\begin{align}
 K_{Ergun}'  &\propto \dfrac{G^{2}}{\rho_{B}A_{across}} \nonumber\\
			  &\propto \dfrac{\dot{m^{2}}/A_{across}^{2}}{\rho_{B}A_{across}} \nonumber \\
			  &\propto \dfrac{\dot{m^{2}}}{\rho_{B}A_{across}^{3}} \nonumber \\
			 K_{Ergun}'  &\propto \dfrac{\dot{m^{2}}}{\rho_{B}D_{Tube}^{6}} \nonumber \nonumber 
\end{align}

Therefore:
\begin{align}
K_{Ergun_{NEW}}^{'} &= \dfrac{\dot{m^{2}}}{\rho_{B}(1.1D_{Tube})^{6}} \nonumber \\
K_{Ergun_{NEW}}^{'} &= \dfrac{\dot{m^{2}}}{1.772\rho_{B}(D_{Tube})^{6}} \nonumber \\
K_{Ergun_{NEW}}^{'} &= 1.772K_{Ergun}^{'} \nonumber
\end{align}


In [120]:
KergNEW = Kerg/1.1**6
KergNEW

0.22578957202151087

In [119]:
1.1**6

1.7715610000000008

In [121]:
def Q2b(var,w):
    FA, FB, FC, P   = var
    Ftot   = FA + FB + FC
    X      = (FAo - FA)/FAo
    T      = To + M*X
    kprime = koprime*exp(-Ea/(R*T))
    Q      = Ftot*R*T/P
    rA     = -kprime*FA/Q
    rB     = -rA
    rC     = -rA
    dFAdW  = rA
    dFBdW  = rB
    dFCdW  = rC
    dPdW   = -KergNEW*(Po/P)*(Ftot/FAo)*(T/To)
    return[dFAdW,dFBdW,dFCdW,dPdW] 

In [129]:
W1 = 299 #keep changing this one
wspan1 = linspace(0,W1,100)
init1 = (FAo,0,0,Po)
sols1 = odeint(Q2b,init1,wspan1)

In [130]:
FA1,FB1,FC1,P1 = sols1.T

XA1 = (FAo - FA1[-1])/FAo

print('conversion=', XA1*100,'%')
print('W = ',W1,'kg catalyst')  
print('P =',P1[-1],'kPa')

conversion= 82.91040603743926 %
W =  299 kg catalyst
P = 150.33114399917594 kPa
